In [3]:
import wave
import numpy as np
import stumpy
import os
from moviepy.editor import *

ModuleNotFoundError: No module named 'stumpy'

In [ ]:
# Used to separate audio from the video. The audio will later be used for synchronization

# Edit the directories for the mp4 files as necessary.

left_dir = r"left.mp4"
right_dir = r"right.mp4"


# Extract audios from the video files. Use .wav because it is lossless.

left = mp.VideoFileClip(left_dir)
left.audio.write_audiofile(r"left.wav")

right= mp.VideoFileClip(right_dir)

right.audio.write_audiofile(r"right.wav")

In [ ]:
# Begin uploading audio from file into a numpy array

left_wav = wave.open("left.wav", "rb")
right_wav = wave.open("right.wav", "rb")

In [ ]:
# Continue uploading audio from file into a numpy array

left_freq = left_wav.getframerate()
right_freq = right_wav.getframerate()

left_samples = left_wav.getnframes()
right_samples = right_wav.getnframes()

left_signal = left_wav.readframes(left_samples)
right_signal = right_wav.readframes(right_samples)

left_audio = left_samples/left_freq
right_audio = right_samples/right_freq

In [ ]:
# Convert into an int16 file. Otherwise, the following functions wont' work (couldn't figure out why)

left_signal_array = np.frombuffer(left_signal, dtype=np.int16)
right_signal_array = np.frombuffer(right_signal, dtype=np.int16)

In [ ]:
# These create the "x" values, or the times. Each time corresponds to a different signal value. 

times_left = np.linspace(0, left_samples/left_freq, num=left_samples)
times_right = np.linspace(0, right_samples/right_freq, num=right_samples)

# Use right channel for left camera and left channel for right camera
left_channel = left_signal_array[1::2]
right_channel = right_signal_array[0::2]


# fft_left = np.fft.fft(left_channel)
# fft_right = np.fft.fft(right_channel)

In [ ]:
# Plot the signal values of the left camera

plt.figure(figsize=(15, 5))
plt.plot(times_left, left_channel)
plt.title('Left Camera')
plt.ylabel('Signal Value')
plt.xlabel('Time (s)')
plt.xlim(0, left_audio)
plt.show()

In [ ]:
# Plot the signal values of the right camera

plt.figure(figsize=(15, 5))
plt.plot(times_right, right_channel)
plt.title('Right Camera')
plt.ylabel('Signal Value')
plt.xlabel('Time (s)')
plt.xlim(0, right_audio)
plt.show()

In [ ]:
# Find the impulse. The impulse should be the highest value in the signal.
# Use the region around the impulse for matrix profile later

left_spike = left_channel.argmax()
right_spike = right_channel.argmax()

left_spike_start = left_spike-10000
left_spike_end = left_spike+10000

right_spike_start = right_spike-10000
right_spike_end = right_spike+10000

In [ ]:
# Plot around the impulse for the left camera

plt.figure(figsize=(15, 5))
plt.plot(times_left, left_channel)
plt.plot(times_left[left_spike_start:left_spike_end], left_channel[left_spike_start:left_spike_end], color="r")
plt.title('Left Camera')
plt.ylabel('Signal Value')
plt.xlabel('Time (s)')
plt.xlim(0, left_audio)
plt.show()

In [ ]:
# Plot around the impulse for the right camera

plt.figure(figsize=(15, 5))
plt.plot(times_right, right_channel)
plt.plot(times_right[right_spike-10000:right_spike+10000], right_channel[right_spike-10000:right_spike+10000], color="r")

plt.title('Right Camera')
plt.ylabel('Signal Value')
plt.xlabel('Time (s)')
plt.xlim(0, right_audio)
plt.show()

In [ ]:
# Prepare the subarrays for the matrix profile algorithm
# The matrix profile algorithm, specifically for conserved pattern detection
# To read more about matrix profiles, visit matrixprofile.org

m = 10000
left_subarray = left_channel[left_spike_start:left_spike_end]
right_subarray = right_channel[right_spike_start:right_spike_end]

In [ ]:
# Matrix profile can find patterns in nlogn time, so we use it to quickly find conserved patterns.

matrix_profile = stumpy.stump(T_A=left_subarray.astype(np.float64), m=m, T_B=right_subarray.astype(np.float64), ignore_trivial=False)

In [ ]:
# Plot the matrix profile

plt.plot(matrix_profile[:,0])
plt.title('Matrix Profile')
plt.ylabel('Value')
plt.xlabel('Subsequence Number')

In [ ]:
# Find the index of th eleft and right motifs

left_motif_index = matrix_profile[:,0].argmin()
right_motif_index = matrix_profile[left_motif_index,1] + right_spike_start
left_motif_index += left_spike_start

In [ ]:
# Plot where the conserved motif occurs in the left audio

plt.figure(figsize=(15, 5))
plt.plot(times_left, left_channel)
plt.plot(times_left[left_motif_index:left_motif_index+m], left_channel[left_motif_index:left_motif_index+m], color="red")
plt.title('Left Camera')
plt.ylabel('Signal Value')
plt.xlabel('Time (s)')
plt.xlim(0, left_audio)
plt.show()

In [ ]:
# Plot where the conserved motif occurs in the right audio

plt.figure(figsize=(15, 5))
plt.plot(times_right, right_channel)
plt.plot(times_right[right_motif_index:right_motif_index+m], right_channel[right_motif_index:right_motif_index+m], color="red")
plt.title('Right Camera')
plt.ylabel('Signal Value')
plt.xlabel('Time (s)')
plt.xlim(0, right_audio)
plt.show()

In [ ]:
# This plotted motif should be the same, if not very simliar to, the consecutive cell

plt.plot(times_right[right_motif_index:right_motif_index+m], right_channel[right_motif_index:right_motif_index+m])
plt.title('Right Camera Most Similar Subsequence')
plt.ylabel('Signal Value')
plt.xlabel('Time (s)')

In [ ]:
# This plotted motif should be the same, if not very similar to, the previous cell

plt.plot(times_left[left_motif_index:left_motif_index+m], left_channel[left_motif_index:left_motif_index+m])
plt.title('Left Camera Most Similar Subsequence')
plt.ylabel('Signal Value')
plt.xlabel('Time (s)')

In [ ]:
# The start and end times for each motif, on the left audio and the right audio

print(times_left[left_motif_index])
print(times_left[-1])
print(times_right[right_motif_index])
print(times_right[-1])

In [ ]:
# Extract the desired cropped left camera video

test = VideoFileClip("left.mp4")
test = test.subclip(times_left[left_motif_index],times_left[-1])

test.ipython_display(filename="left_trimmed.mp4", width=360)
os.rename("__temp__.mp4", "left_trimmed.mp4")

In [ ]:
# Extract the desired cropped right camera video

test = VideoFileClip("right.mp4")
test = test.subclip(times_right[right_motif_index],times_right[-1])

test.ipython_display(filename="right_trimmed.mp4", width=360)
os.rename("__temp__.mp4", "right_trimmed.mp4")